In [5]:
import json
import pandas as pd
from collections import defaultdict
import evaluate

# Load metrics
metric_rouge = evaluate.load('rouge')
metric_meteor = evaluate.load('meteor')

# Define function to search for COVID related data
def contains_covid_related(text, covid_keywords):
    text = text.lower()
    return any(keyword in text for keyword in covid_keywords)

# Define function to compute metrics
def compute_metrics(data, predictions):
    #references = [entry['exact_answer'] for entry in data]
    #references = [predictions.loc[idx]['reference'] for idx in data.index]
    references = [eval(predictions.loc[idx]['reference'])[0] for idx in data.index]

    model_outputs = [predictions.loc[idx]['model_prediction'] for idx in data.index]
    
    # Compute scores
    rouge_scores = metric_rouge.compute(predictions=model_outputs, references=references)
    meteor_scores = metric_meteor.compute(predictions=model_outputs, references=references)

    return {
        "rougeL": rouge_scores['rougeL'],
        "meteor": meteor_scores['meteor']
    }

# Paths and settings
json_paths = "/home/weisi/TemporalAssessment/data/BIOASQ_formatted/alltypes/seed{}/bioasq-alltypes-T4_2021_2022-test.json"
predict_results_path = "/HDD16TB/weisi/logs/BioASQ_alltypes_exact_new/t5-base/{}/split_seed{}/b4a1_sd42_3e-4_maxanslen30_20epc/predict_results.txt"
pairs = ["T1_T4", "T2_T4", "T3_T4", "T4_T4"]
seeds = range(1, 6)
covid_keywords = ["covid", "coronavirus", "sars-cov-2"]

# Process data
results = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for pair in pairs:
    for seed in seeds:
        # Load JSON data
        with open(json_paths.format(seed)) as f:
            data = [json.loads(line) for line in f]
        df_data = pd.DataFrame(data)
        # Identify COVID-related entries
        df_data['covid_related'] = df_data.apply(lambda row: contains_covid_related(
            f"{row['body']} {row['exact_answer']} {row['snippets']}", covid_keywords), axis=1)
        covid_data = df_data[df_data['covid_related']]
        non_covid_data = df_data[~df_data['covid_related']]

        # Load prediction results
        predict_df = pd.read_csv(predict_results_path.format(pair, seed), delimiter="\t")

        # Calculate metrics for COVID and non-COVID data
        results[pair]['covid'][seed] = compute_metrics(covid_data, predict_df)
        results[pair]['non_covid'][seed] = compute_metrics(non_covid_data, predict_df)
        results[pair]['all'][seed] = compute_metrics(df_data, predict_df)  # All data

# Calculate averages over seeds
final_results = defaultdict(dict)
for pair in pairs:
    for category in ['covid', 'non_covid', 'all']:
        avg_rougeL = sum(results[pair][category][seed]['rougeL'] for seed in seeds) / len(seeds)
        avg_meteor = sum(results[pair][category][seed]['meteor'] for seed in seeds) / len(seeds)
        final_results[pair][category] = {'rougeL': avg_rougeL, 'meteor': avg_meteor}

print(final_results)


[nltk_data] Downloading package wordnet to /home/weisi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/weisi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/weisi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


defaultdict(<class 'dict'>, {'T1_T4': {'covid': {'rouge1': 0.37632694896150853, 'meteor': 0.20440979597022407}, 'non_covid': {'rouge1': 0.37467474852719923, 'meteor': 0.26016716321668276}, 'all': {'rouge1': 0.3756272361866352, 'meteor': 0.25819647182358185}}, 'T2_T4': {'covid': {'rouge1': 0.352104000854357, 'meteor': 0.1926545939867306}, 'non_covid': {'rouge1': 0.39090871447322995, 'meteor': 0.2684253082447878}, 'all': {'rouge1': 0.39027048247286167, 'meteor': 0.26577546574238864}}, 'T3_T4': {'covid': {'rouge1': 0.3880022703708249, 'meteor': 0.20951065332662075}, 'non_covid': {'rouge1': 0.41816935824916646, 'meteor': 0.2865614094565391}, 'all': {'rouge1': 0.418577266738186, 'meteor': 0.2840456925328139}}, 'T4_T4': {'covid': {'rouge1': 0.40351964734309764, 'meteor': 0.2591697059596233}, 'non_covid': {'rouge1': 0.41652747973944715, 'meteor': 0.28998361693689356}, 'all': {'rouge1': 0.4158577175985575, 'meteor': 0.28835192056003744}}})


In [5]:
defaultdict(<class 'dict'>, {'T1_T4': {'covid': {'rouge1': 0.3715989016485074, 'meteor': 0.03472468091264295}, 
                                       'non_covid': {'rouge1': 0.36553537663449714, 'meteor': 0.07354297295956666}, 
                                       'all': {'rouge1': 0.3655229755000633, 'meteor': 0.07203071419739235}}, 
                            'T2_T4': {'covid': {'rouge1': 0.34066589560047783, 'meteor': 0.03881677626364134}, 
                                                 'non_covid': {'rouge1': 0.3756740030830755, 'meteor': 0.07729882869096893}, 
                                                 'all': {'rouge1': 0.37390418660549735, 'meteor': 0.07602136315017098}}, 
                            'T3_T4': {'covid': {'rouge1': 0.39170355072626467, 'meteor': 0.038831243367532665}, 
                                      'non_covid': {'rouge1': 0.4040374648648948, 'meteor': 0.07844993756553685}, 
                                      'all': {'rouge1': 0.4037457804642588, 'meteor': 0.07703786356536013}}, 
                            'T4_T4': {'covid': {'rouge1': 0.3955089895700691, 'meteor': 0.05307968231256249}, 
                                      'non_covid': {'rouge1': 0.40549076877297086, 'meteor': 0.08405589117564437}, 
                                      'all': {'rouge1': 0.4053676929629974, 'meteor': 0.08268101377254067}}})


defaultdict(<class 'dict'>, {'T1_T4': {'covid': {'rouge1': 0.3715989016485074, 'meteor': 0.03472468091264295}, 'non_covid': {'rouge1': 0.36553537663449714, 'meteor': 0.07354297295956666}, 'all': {'rouge1': 0.3655229755000633, 'meteor': 0.07203071419739235}}, 'T2_T4': {'covid': {'rouge1': 0.34066589560047783, 'meteor': 0.03881677626364134}, 'non_covid': {'rouge1': 0.3756740030830755, 'meteor': 0.07729882869096893}, 'all': {'rouge1': 0.37390418660549735, 'meteor': 0.07602136315017098}}, 'T3_T4': {'covid': {'rouge1': 0.39170355072626467, 'meteor': 0.038831243367532665}, 'non_covid': {'rouge1': 0.4040374648648948, 'meteor': 0.07844993756553685}, 'all': {'rouge1': 0.4037457804642588, 'meteor': 0.07703786356536013}}, 'T4_T4': {'covid': {'rouge1': 0.3955089895700691, 'meteor': 0.05307968231256249}, 'non_covid': {'rouge1': 0.40549076877297086, 'meteor': 0.08405589117564437}, 'all': {'rouge1': 0.4053676929629974, 'meteor': 0.08268101377254067}}})


In [ ]:

defaultdict(<class 'dict'>, {'T1_T4': {'covid': {'rouge1': 0.37814070691213547, 'meteor': 0.20440979597022407}, 
                                        'non_covid': {'rouge1': 0.38690883338019516, 'meteor': 0.26016716321668276}, 
                                            'all': {'rouge1': 0.3878181283630212, 'meteor': 0.25819647182358185}}, 
                            'T2_T4': {'covid': {'rouge1': 0.35595463984047326, 'meteor': 0.1926545939867306}, 
                                        'non_covid': {'rouge1': 0.397610578597544, 'meteor': 0.2684253082447878}, 
                                            'all': {'rouge1': 0.39743232482815555, 'meteor': 0.26577546574238864}}, 
                            'T3_T4': {'covid': {'rouge1': 0.4000567302390974, 'meteor': 0.20951065332662075}, 
                                        'non_covid': {'rouge1': 0.4265462800302807, 'meteor': 0.2865614094565391}, 
                                            'all': {'rouge1': 0.4265728015599642, 'meteor': 0.2840456925328139}}, 
                            'T4_T4': {'covid': {'rouge1': 0.40093087459822724, 'meteor': 0.2591697059596233}, 
                                        'non_covid': {'rouge1': 0.4268519079362204, 'meteor': 0.28998361693689356}, 
                                            'all': {'rouge1': 0.42651020631886344, 'meteor': 0.28835192056003744}}})
0.37632694896150853
0.352104000854357
0.3880022703708249
0.418577266738186
0.204,0.193,0.210,0.259

In [ ]:

defaultdict(<class 'dict'>, {'T1_T4': {'covid': {'rouge1': 0.37632694896150853, 'meteor': 0.20440979597022407}, 'non_covid': {'rouge1': 0.37467474852719923, 'meteor': 0.26016716321668276}, 'all': {'rouge1': 0.3756272361866352, 'meteor': 0.25819647182358185}}, 'T2_T4': {'covid': {'rouge1': 0.352104000854357, 'meteor': 0.1926545939867306}, 'non_covid': {'rouge1': 0.39090871447322995, 'meteor': 0.2684253082447878}, 'all': {'rouge1': 0.39027048247286167, 'meteor': 0.26577546574238864}}, 'T3_T4': {'covid': {'rouge1': 0.3880022703708249, 'meteor': 0.20951065332662075}, 'non_covid': {'rouge1': 0.41816935824916646, 'meteor': 0.2865614094565391}, 'all': {'rouge1': 0.418577266738186, 'meteor': 0.2840456925328139}}, 'T4_T4': {'covid': {'rouge1': 0.40351964734309764, 'meteor': 0.2591697059596233}, 'non_covid': {'rouge1': 0.41652747973944715, 'meteor': 0.28998361693689356}, 'all': {'rouge1': 0.4158577175985575, 'meteor': 0.28835192056003744}}})
